矩阵分解常用的方法有：
   - SVD
   - NMF
   - RSVD
   - SVD++
   - SVDfeature
   - Libmf/Libfm

对用户-物品评分矩阵进行分解，分解成若干个小矩阵，目的是分解之后的矩阵乘积接近于原始矩阵，于是也实现了对原始矩阵空值的预测。这些方法中，比较重要的参数有隐特征个数、随机梯度下降中的学习率、正则化参数，迭代次数等。每个方法中参数的最优值也不同。

这里简单利用surprise调用svd++

In [111]:
import os 
import numpy as np
import pandas as pd
from surprise import SVD, SVDpp
from surprise import Dataset, Reader
from surprise import evaluate, print_perf

# 数据预处理

In [112]:
# 加载训练集、测试集数据
train = pd.read_csv('../data/train.csv')
test = pd.read_csv('../data/test.csv')

In [113]:
train.head()

,uid,iid,score,time
0,0,0,2,19
1,0,8,4,273
2,0,13,1,587
3,0,18,3,15
4,0,34,3,17


In [123]:
# 取部分数据集
train = train[train['uid']<10]
test = test[test['uid']<5]

In [128]:
train.shape

(8709, 4)

In [129]:
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(train[['uid', 'iid', 'score']], reader)
data.split(2)

In [130]:
# 使用SVD算法
algo = SVDpp()

# 在数据集上评估算法表现
performance = evaluate(algo, data, measures=['RMSE', 'MAE'])
print_perf(performance)

Evaluating RMSE, MAE of algorithm SVDpp.

------------
Fold 1
RMSE: 1.0653
MAE:  0.8456
------------
Fold 2
RMSE: 1.0726
MAE:  0.8594
------------
------------
Mean RMSE: 1.0690
Mean MAE : 0.8525
------------
------------
        Fold 1  Fold 2  Mean    
RMSE    1.0653  1.0726  1.0690  
MAE     0.8456  0.8594  0.8525  


In [133]:
def predict():
    """ 预测 """
    num = test.shape
    result = np.zeros(num[0])
    for i in range(num[0]):
        a = test.ix[i, 'uid']
        b = test.ix[i, 'iid']
        result[i] = algo.predict(a, b).est
    return result

In [134]:
result = predict()

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  import sys


In [135]:
Y_test_score = pd.DataFrame(np.array(result), columns=['score'])

In [137]:
Y_test_score

,score
0,2.798981
1,2.692117
2,3.116730
3,2.834200
4,2.598298
5,2.970516
6,2.834200
7,2.834200
8,2.900247
9,3.607066
